## 🧰 Initial Settings

### 📦 Import Libraries

In [1]:
import os, glob, shutil, random, cv2
from ultralytics import YOLO
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

### 📥 Download Dataset from Kaggle

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

DATASET_DIR = "Face emotion datasets"
DATASET_SLUG = "hhshamimkhan/face-emotion-datasets"

if not os.path.exists(DATASET_DIR):
    os.environ["KAGGLE_CONFIG_DIR"] = os.getcwd()
    api = KaggleApi()
    api.authenticate()

    api.dataset_download_files(DATASET_SLUG, unzip=True)

## 🛠️ Dataset Preparation for YOLO

### ⚙️ Dataset Paths, Split Ratios, and Configuration Settings

In [3]:
SRC_ROOT = "Face emotion datasets"
OUT_ROOT = "dataset_yolo"
R_TRAIN, R_VAL = 0.70, 0.15
MAX_PER_CLASS_TOTAL = 1000
SEED = 42
VALID_EXT = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

random.seed(SEED)

### 📊 Count Images per Class and Total Samples

In [4]:
counts = {}
for cls in sorted(
    d for d in os.listdir(SRC_ROOT) if os.path.isdir(os.path.join(SRC_ROOT, d))
):
    n = len(
        [
            p
            for p in glob.glob(os.path.join(SRC_ROOT, cls, "*"))
            if p.lower().endswith(VALID_EXT)
        ]
    )
    counts[cls] = n

for k, v in counts.items():
    print(f"{k}: {v}")
print("Total:", sum(counts.values()))

Anger: 2310
Content: 772
Disgust: 788
Fear: 906
Happy: 4929
Neutral: 5016
Sad: 4771
SillyFace: 1116
Surprise: 2231
Total: 22839


### 🔎 Discover Classes and Assign IDs

In [5]:
classes = sorted(
    [d for d in os.listdir(SRC_ROOT) if os.path.isdir(os.path.join(SRC_ROOT, d))]
)
name2id = {c: i for i, c in enumerate(classes)}
print("Clases -> IDs:")
name2id

Clases -> IDs:


{'Anger': 0,
 'Content': 1,
 'Disgust': 2,
 'Fear': 3,
 'Happy': 4,
 'Neutral': 5,
 'Sad': 6,
 'SillyFace': 7,
 'Surprise': 8}

### 🌐 Download YuNet ONNX Model

In [6]:
import urllib.request

yunet_file = "face_detection_yunet_2023mar.onnx"

if not os.path.exists(yunet_file):
    url = "https://github.com/opencv/opencv_zoo/raw/main/models/face_detection_yunet/face_detection_yunet_2023mar.onnx"
    urllib.request.urlretrieve(url, yunet_file)

### 👤 Initialize Haar Cascade and YuNet Face Detectors

In [7]:
haar = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

yunet_model = cv2.FaceDetectorYN.create(
    model=yunet_file,
    config="",
    input_size=(640, 640),
    score_threshold=0.5,
    nms_threshold=0.3,
    top_k=5000,
)

### 🧩 Face Detection & Labeling Helpers (YuNet/Haar + YOLO Norm)

In [ ]:
def detect_faces_bgr(img_bgr):
    H, W = img_bgr.shape[:2]

    # Try YuNet
    try:
        yunet_model.setInputSize((W, H))
        faces = yunet_model.detect(img_bgr)[1]
        if faces is not None and len(faces) > 0:
            boxes = []
            for row in faces:
                x, y, w, h = row[:4]
                boxes.append([int(x), int(y), int(w), int(h)])
            return boxes
    except Exception:
        pass

    # Haar Cascade
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray_eq = clahe.apply(gray)
    faces = haar.detectMultiScale(
        gray_eq, scaleFactor=1.2, minNeighbors=5, minSize=(60, 60)
    )
    return [[int(x), int(y), int(w), int(h)] for (x, y, w, h) in faces]


def box_to_yolo_norm(x, y, w, h, W, H):
    xc = (x + w / 2.0) / W
    yc = (y + h / 2.0) / H
    ww = w / W
    hh = h / H
    # clip
    xc = min(max(xc, 0.0), 1.0)
    yc = min(max(yc, 0.0), 1.0)
    ww = min(max(ww, 0.0), 1.0)
    hh = min(max(hh, 0.0), 1.0)
    return xc, yc, ww, hh


def load_image_or_create_empty_label(src_img, out_root, split, fn):
    img_bgr = cv2.imread(src_img)

    if img_bgr is None:
        lbl_path = os.path.join(
            out_root, "labels", split, os.path.splitext(fn)[0] + ".txt"
        )
        open(lbl_path, "w").close()
        return None, None, None

    H, W = img_bgr.shape[:2]
    return img_bgr, H, W


def select_faces_by_area(boxes, W, H, min_rel_area=0.03, top_k=1):
    if not boxes:
        return []

    total_area = float(W * H)
    areas = [b[2] * b[3] for b in boxes]
    threshold = min_rel_area * total_area

    # Select boxes with area >= threshold
    large = [b for b, a in zip(boxes, areas) if a >= threshold]

    if large:
        large_sorted = sorted(large, key=lambda b: b[2]*b[3], reverse=True)
        return large_sorted[:top_k]

    biggest = max(boxes, key=lambda b: b[2]*b[3])
    return [biggest]


### 🖼️ Split Data, Detect Faces, and Generate YOLO Labels


In [9]:
for s in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUT_ROOT, "images", s), exist_ok=True)
    os.makedirs(os.path.join(OUT_ROOT, "labels", s), exist_ok=True)

for cls in classes:
    imgs = [
        p
        for p in glob.glob(os.path.join(SRC_ROOT, cls, "*"))
        if p.lower().endswith(VALID_EXT)
    ]
    random.shuffle(imgs)
    imgs = imgs[: min(len(imgs), MAX_PER_CLASS_TOTAL)]

    n = len(imgs)
    n_tr = int(n * R_TRAIN)
    n_va = int(n * R_VAL)

    splits = [
        ("train", imgs[:n_tr]),
        ("val", imgs[n_tr : n_tr + n_va]),
        ("test", imgs[n_tr + n_va :]),
    ]

    for split, items in splits:
        for src_img in items:
            fn = os.path.basename(src_img)
            dst_img = os.path.join(OUT_ROOT, "images", split, fn)
            
            lbl_path = os.path.join(OUT_ROOT, "labels", split, os.path.splitext(fn)[0] + ".txt")
            if os.path.exists(lbl_path):
                if not os.path.exists(os.path.join(OUT_ROOT, "images", split, fn)):
                    shutil.copy2(src_img, os.path.join(OUT_ROOT, "images", split, fn))
                continue
            
            shutil.copy2(src_img, dst_img)

            img_bgr, H, W = load_image_or_create_empty_label(
                src_img, OUT_ROOT, split, fn
            )
            if img_bgr is None:
                continue

            boxes = detect_faces_bgr(img_bgr)
            boxes = select_faces_by_area(boxes, W, H, min_rel_area=0.03, top_k=1) 

            with open(lbl_path, "w") as f:
                for (x, y, w, h) in boxes:
                    xc, yc, ww, hh = box_to_yolo_norm(x, y, w, h, W, H)
                    f.write(f"{name2id[cls]} {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}\n")

### 📸 Count Images per Split (Train/Val/Test)


In [10]:
for split in ["train", "val", "test"]:
    img_dir = os.path.join(OUT_ROOT, "images", split)
    n = len([f for f in os.listdir(img_dir) if f.lower().endswith(VALID_EXT)])
    print(split, "→", n, "images")

train → 5925 images
val → 1268 images
test → 1273 images


### 📄 Create YOLO Configuration File (data.yaml)

In [11]:
with open(os.path.join(OUT_ROOT, "data.yaml"), "w", encoding="utf-8") as f:
    f.write(
        f"path: {OUT_ROOT}\n"
        f"train: images/train\n"
        f"val: images/val\n"
        f"test: images/test\n"
        f"names: [{', '.join(classes)}]\n"
    )
print("Clases:", classes)

Clases: ['Anger', 'Content', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'SillyFace', 'Surprise']


## 🤖 Train YOLO Model

### 📦 Load Pre-trained YOLO Model

In [12]:
yolo = YOLO("yolo11n.pt")

### 🚀 Training YOLO Model on Dataset

In [13]:
yolo.train(
    data="dataset_yolo/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    cos_lr=True,
)

New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.201 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=dataset_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, ops

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4c67b1e480>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

## 📏 Evaluate YOLO Model on Test Set


In [20]:
metrics = yolo.val(split="test")
print(metrics)

Ultralytics 8.3.201 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 77.2±26.9 MB/s, size: 28.3 KB)
val: Scanning /home/garayco/maestria/deep_learning/proyecto_2/dataset_yolo/labels/test.cache... 1273 images, 4 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1273/1273 680.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 80/80 7.4it/s 10.8s0.1s
                   all       1273       1269      0.777      0.779      0.832       0.66
                 Anger        150        150      0.709       0.72      0.783      0.609
               Content        117        117      0.775      0.855      0.865      0.709
               Disgust        119        119      0.682      0.832      0.815      0.615
                  Fear        137        137      0.811      0.825      0.884      0.682
                 Happy        150        150      0.877      0.827

In [15]:
best_model = YOLO("runs/detect/train/weights/best.pt")

In [19]:
import ipywidgets as w


def show_pred(x, conf=0.25, iou=0.7, title=None):
    r = best_model.predict(x, conf=conf, iou=iou)[0]
    plt.figure(figsize=(7, 5))
    plt.imshow(r.plot())
    if title:
        plt.title(title)
    plt.axis("off")
    plt.show()


TEST_DIR = "dataset_yolo/images/test"
test_imgs = [p for p in glob.glob(os.path.join(TEST_DIR, "*"))]

uploader = w.FileUpload(accept="image/*", multiple=False)
btn_random = w.Button(description="🎲 Random from test", button_style="primary")
conf_s = w.FloatSlider(description="conf", value=0.25, min=0.05, max=0.95, step=0.05)
iou_s = w.FloatSlider(description="iou", value=0.70, min=0.10, max=0.95, step=0.05)
out = w.Output()

ui = w.VBox(
    [
        w.HTML("<h3>Facial Emotion Recognition (YOLO)</h3>"),
        w.HBox([btn_random, uploader]),
        w.HBox([conf_s, iou_s]),
        out,
    ]
)
display(ui)


def _get_uploaded_bytes(upl):
    v = upl.value
    if not v:
        return None
    fileinfo = list(v.values())[0] if isinstance(v, dict) else v[0]
    data = fileinfo["content"]
    return data.tobytes() if isinstance(data, memoryview) else data


def on_random(_):
    print("on_random_fired")
    out.clear_output()
    if not test_imgs:
        with out:
            print("No test images found.")
            return
    path = random.choice(test_imgs)
    with out:
        show_pred(
            path, conf=conf_s.value, iou=iou_s.value, title=os.path.basename(path)
        )


def on_upload(_):
    out.clear_output()
    data = _get_uploaded_bytes(uploader)
    if data is None:
        with out:
            print("Upload an image first.")
            return
    img = Image.open(BytesIO(data)).convert("RGB")
    with out:
        show_pred(img, conf=conf_s.value, iou=iou_s.value)


btn_random.on_click(on_random)
uploader.observe(on_upload, names="value")